In [147]:
import sqlite3 as sql
import pandas as pd

conn = sql.connect("ncaa_pbp.db")
df = pd.read_sql("""
                    SELECT
                        EventPlayerID, EventTeamID, EventType, EventID
                    From
                        test
                    Limit
                        1000000000
                     """, conn, index_col='EventID')


In [148]:
pointVals = {'miss2_dunk': 0, 
             'made2_lay': 2, 
             'miss3_jump': 0,
             'made2_dunk': 2,
             'made2_tip_and1': 3,
             'miss2_jump': 0,
             'made2_dunk_and0': 2,
             'made2_jump_and0': 2,
             'made3_jump_and1': 4,
             'made2_lay_and0': 2,
             'made3_jump_and0': 3,
             'made2_jump_and1': 3,
             'made3_jump': 3,
             'miss2_tip': 0,
             'made2_jump': 2,
             'made2_tip': 2,
             'made2_tip_and0': 2,
             'made2_dunk_and1': 3,
             'miss2_lay': 0,
             'ft_0_1': 0,
             'ft_0_2': 0,
             'ft_0_3': 0,
             'ft_1_1': 1,
             'ft_1_2': 1,
             'ft_1_3': 1,
             'ft_2_2': 2,
             'ft_2_3': 2,
             'ft_3_3': 3,
             'made2_lay_and1': 3}


def applyPointValues(df):
    df['PointValue'] = df['EventType'].apply(lambda x: pointVals[x])
    
    return df

In [149]:
df = applyPointValues(df)

In [150]:
df.reset_index(inplace=True)
df.sort_values(['EventTeamID', 'EventID'], inplace=True)
df = df.join(df.shift(-1), rsuffix='_sbsq')

In [151]:
df.drop(['EventType_sbsq', 'EventPlayerID_sbsq', 'EventID_sbsq'], axis=1, inplace=True)

In [152]:
df = df[df['EventTeamID'] == df['EventTeamID_sbsq']]

In [153]:
df = df[['EventPlayerID', 'EventType', 'PointValue_sbsq']]

In [154]:
names = pd.read_sql("""Select PlayerID, PlayerName FROM players WHERE Season = 2018""", conn)

In [155]:
def tidy_names(name):
    name = name.split('_')
    first = name[-1]
    last = ' '.join(name[:-1])
    name = first.title() + ' ' + last.title()

    return name

In [156]:
df.set_index('EventPlayerID', inplace=True)
names['PlayerName'] = names['PlayerName'].apply(tidy_names)
names.set_index('PlayerID', inplace=True)
df = df.join(names)

In [157]:
names

,PlayerName
PlayerID,
648095,Chase Cobb
648096,Hayden Farqhar
648097,Hayden Farquhar
648098,Tevin Foster
648099,Jaylen Franklin
648100,Jalone Friday
648101,Drake Green
648102,Hayden Howell
648103,Kolton Kohl


In [158]:
mean = df.groupby(['PlayerName', 'EventType']).mean()
mean.columns = ['Average Points']
count = df.groupby(['PlayerName', 'EventType']).count()
count.columns = ['Count']

In [173]:
df = mean.join(count)

In [174]:
df = df[df['Count'] > 50]
df = df.sort_values('Average Points', ascending=False)

In [175]:
from collections import Counter
Counter(df.reset_index()['PlayerName']).most_common()

[('Jordan Howard', 7),
 ('Mike Daum', 7),
 ('Jerome Robinson', 7),
 ('Rob Gray', 7),
 ('Jevon Carter', 7),
 ('Tra Holder', 6),
 ('Kevin Hervey', 6),
 ('Jerrick Harding', 6),
 ('Bogdan Bliznyuk', 6),
 ('Aaron Holiday', 6),
 ('Tyler Nelson', 6),
 ('Donte Clark', 6),
 ('Cj Burks', 6),
 ('Zach Thomas', 6),
 ('Jon Elmore', 6),
 ('Collin Sexton', 6),
 ('Joel Hernandez', 6),
 ('Trae Young', 6),
 ('Junior Robinson', 6),
 ('Kelan Martin', 6),
 ('Tanner Leissner', 6),
 ('Andre Spight', 6),
 ('Jairus Lyles', 6),
 ('Shavar Newkirk', 6),
 ('Gary Blackston', 6),
 ('Rj Cole', 6),
 ('Kj Feagin', 6),
 ('Otis Livingston Ii', 6),
 ('Treshaun Fletcher', 5),
 ('Justin Tillman', 5),
 ('Matt Mooney', 5),
 ('Roddy Peters', 5),
 ('Devonte Graham', 5),
 ('Jaylen Barford', 5),
 ('Trevon Bluiett', 5),
 ('Darius Thompson', 5),
 ('Kahlil Dukes', 5),
 ('Jalen Brunson', 5),
 ('Kamar Baldwin', 5),
 ('Jaelan Sanford', 5),
 ('J Wright-Foreman', 5),
 ('Milik Yarbrough', 5),
 ('Anthony Cowan', 5),
 ('Peyton Aldridge', 5),

In [180]:
df.loc['Collin Sexton']

,Average Points,Count
EventType,,
miss3_jump,1.287356,87
miss2_jump,1.187500,96
made2_jump,1.113208,53
miss2_lay,1.107143,56
made2_lay,0.926829,82
ft_2_2,0.904110,73
